# Credit Card Fraud Prediction - Feature Engineering

This demo is based on the Machine Learning for Credit Card Fraud detection - Practical handbook, https://fraud-detection-handbook.github.io/fraud-detection-handbook/

## Baseline feature transformation

### Import Snowpark & Connect to Snowflake

Here we first import the Snowpark dependencies and then make a connection with the Snowflake account.

The creds.json file used has information about what Snowflake Account to use, User, Database and what Virtual Warehouse to use for this session. 

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import *
from snowflake.snowpark import Window

import json

In [ ]:
with open('creds.json') as f:
    connection_parameters = json.load(f)    

Connect to Snowflake using the parameters in creds.json

In [ ]:
session = Session.builder.configs(connection_parameters).create()

### Define a DataFrame

We start by defining a Snowpark Dataframe that reference the **CUSTOMER_TRANSACTIONS_FRAUD** table in our database. No data will be pulled back and the **dfCustTrxFraud** is basicly only containing the SQL needed to use the table. The below image gives a illustration on what type of data it has.

![Original Data Frame](images/figure1.png)

Using the **show** command brings back 10 rows to the client, meaning the SQL to use the table are executed in Snowflake.

In [ ]:
dfCustTrxFraud = session.table("CUSTOMER_TRANSACTIONS_FRAUD")
dfCustTrxFraud.show()

## Data Understanding

Let's start by getting some basic understanding of our data.

We can use the **describe** function on our **numeric** columns to get some basic statistics. The only column in our current dataset would be TX_AMOUNT.

In [ ]:
dfCustTrxFraud.describe(["TX_AMOUNT"]).show()

By aggregating on date and count the number of rows we can visulaize over time, plotly is a good library to do that and inb order to use it we need to pull back our reulst to the client which can be done with **toPandas**

In [ ]:
import plotly.express as px

dfCountbyDate = dfCustTrxFraud.groupBy(to_date(col("TX_DATETIME"))).count().sort(col("TO_DATE(TX_DATETIME)")).select(col("TO_DATE(TX_DATETIME)").as_("DATE"), col("COUNT")).toPandas()
fig = px.line(dfCountbyDate, x="DATE", y="COUNT", title='Transactions by Day')
fig.show()

We can see that we have about 6 months  of transactions and around 9k transactions/day.

Let's count the number of fraudulent and none fraudulent transactions, by using the **call_builtin** function we can also use the use **RATIO_TO_REPORT** function (that currently is not exposed in the Snowpark API) to also get precentages

In [ ]:
dfCustTrxFraud.groupBy(col("TX_FRAUD")).agg(count(col("TRANSACTION_ID")).as_("NB_TX_DAY"))\
            .select(col("TX_FRAUD"), col("NB_TX_DAY"), call_builtin("RATIO_TO_REPORT", col("NB_TX_DAY")).over() * 100 )\
            .show()

We can see that about 0.9% of all transactions are fraud.

If we look at the number of fraudlent transactions and unique cards used we can see that most cards is used very few times.

In [ ]:
pdDayByCard = dfCustTrxFraud.filter(col("TX_FRAUD") == 1)\
                             .groupBy(to_date(col("TX_DATETIME"))).agg([sum(col("TX_FRAUD")).as_("NBR_FRAUD_TRX"), count_distinct(col("CUSTOMER_ID")).as_("NBR_FRAUD_CARD")])\
                             .sort(col("TO_DATE(TX_DATETIME)")).toPandas()
fig = px.line(pdDayByCard, x="TO_DATE(TX_DATETIME)", y=["NBR_FRAUD_TRX", "NBR_FRAUD_CARD"], title='Fraud Transactions vs Fraud Cards')
fig.show()


## Feature Engineering
### Date and time transformations

We will create two new binary features from the transaction dates and times:

* The first will characterize whether a transaction occurs during a weekday (value 0) or a weekend (1), and will be called TX_DURING_WEEKEND
* The second will characterize whether a transaction occurs during the day (0) or during the night (1). The night is defined as hours that are between 20pm and 6am. It will be called TX_DURING_NIGHT.

This can be done using the built in date functions in Snowflake that are exposed in the Snowpark API and will be based on the **TX_DATETIME** column, as illustrated in the image below.

![Figure2](images/figure2.png)

In [ ]:
dfDateTimeFeat = dfCustTrxFraud.withColumns(["TX_DURING_WEEKEND",  "TX_DURING_NIGHT"],\
                                            [iff((call_builtin("dayofweek", col("TX_DATETIME")) == lit(6)) | (call_builtin("dayofweek", col("TX_DATETIME")) == lit(0)), lit(1), lit(0)),\
                                             iff((call_builtin("hour", col("TX_DATETIME")) < lit(6)) | (call_builtin("hour",col("TX_DATETIME")) > lit(20)), lit(1), lit(0))])

dfDateTimeFeat.sort(col("TRANSACTION_ID")).show()

Is there differences between the number of fraud cases during weekdays/weekdays and day/Night?

In [ ]:
window = Window.partitionBy(col("TX_DURING_WEEKEND"), col("TX_DURING_NIGHT"))

dfDateTimeFeat.groupBy(col("TX_DURING_WEEKEND"),col("TX_DURING_NIGHT"),col("TX_FRAUD"))\
                    .count().sort(col("TX_DURING_WEEKEND"), col("TX_DURING_NIGHT"), col("TX_FRAUD"))\
                    .select(col("TX_DURING_WEEKEND"), col("TX_DURING_NIGHT"), col("TX_FRAUD"), call_builtin("RATIO_TO_REPORT", col("COUNT")).over(window) * 100 )\
                    .sort(col("TX_DURING_WEEKEND"), col("TX_DURING_NIGHT"), col("TX_FRAUD")).show()

We can see that basicly the number of fradulent transactions are the same during the days

### Customer spending behaviour transformations

We will compute two customer spending behaviour features.

The first feature will be the number of transactions that occur within a time window (Frequency). The second will be the average amount spent in these transactions (Monetary value). The time windows will be set to one, seven, and thirty days. 

The values is to be calculated based on day level where our transactions is on seconds level, the table below show a example of the output for the 1 day window.

|CUSTOMER_ID|TX_DATETIME|TX_AMOUNT|AVG 1 DAY WIN|NBR TRX  1 DAY WIN|
|:---|:---|:---|:---|:---|
|0|2019-04-01 07:19:05.000|123.59|123.59|1|
|0|2019-04-01 18:00:16.000|77.34|100.465|2|
|0|2019-04-01 19:02:02.000|46.51|82.48|3|
|0|2019-04-02 08:51:06.000|54.72|59.523333333|4|
|0|2019-04-02 14:05:38.000|63.3|60.4675|5|
|0|2019-04-02 15:13:02.000|32.35|54.844|6|
|0|2019-04-02 15:46:51.000|13.59|47.968333333|7|
|0|2019-04-02 20:24:47.000|51.89|43.17|8|
|0|2019-04-03 07:41:24.000|93.26|51.518333333|6|

Since we want to aggregate by day and also take include dates that has no transactions (so our windows are real days) we need to first create a new data frame that has for each customer one row for each date between the minimum transaction date and maximum transaction date. Snowpark has a function, **range** , that can be used to generate n number of rows. Since we want to generate a row for each date between minimum and maximum we need to calculate that first. Once we have that dataframe we can do a cross join with our **CUSTOMER** table to create a new dataframe that has now one row for each date between the minimum transaction date and maximum transaction date and customer, as illustrated in the image below.

![Figure3](images/figure3.png)

In [ ]:

dateInfo = dfCustTrxFraud.select(min(col("TX_DATETIME")).as_("END_DATE"), datediff("DAY", col("END_DATE"), max(col("TX_DATETIME"))).as_("NO_DAYS")).toPandas()
nDays = int(dateInfo['NO_DAYS'].values[0])
dStartDate = str(dateInfo['END_DATE'].values[0].astype('datetime64[D]'))

# Create a dataframe with one row for each date between the min and max transaction date
dfDays = session.range(nDays).withColumn("TX_DATE", to_date(dateadd("DAY", call_builtin("SEQ4"), lit(dStartDate))))

# Since we aggregate by customer and day and not all customers have transactions for all dates we cross join our date dataframe with our 
# customer table so each customer witll have one row for each date
dfCustomers = session.table("CUSTOMERS").select("CUSTOMER_ID")
dfCustDay = dfDays.join(dfCustomers)

In [ ]:
dfCustDay.show()

We can now use the new data frame, **dfCustDay**, to aggregate the number of transaction and transaction amount by day, for days that a customer has no transaction we will use 0. The picture below illustrates what we are doing.

Earlier in the data understanding part we used **call_builtin** to use a Snowflake functions that is not exposed in the SNowpark API, we can also use **builtin** to assign the function to a variable that we then can use instead how having to write callBiltin each time.



![Figure4](images/figure4.png)

In [ ]:
zeroifnull = builtin("ZEROIFNULL") # Snowpark does not expose ZEROIFNULL but we can use the callBuiltin function for that or the builtin to add it to a variable that can be used in our code

dfCustTrxByDay = dfCustTrxFraud.join(dfCustDay, (dfCustTrxFraud.col("CUSTOMER_ID") == dfCustDay.col("CUSTOMER_ID")) & (to_date(dfCustTrxFraud.col("TX_DATETIME")) == dfCustDay.col("TX_DATE")), "rightouter") \
            .select(dfCustDay.col("CUSTOMER_ID").as_("CUSTOMER_ID"),\
                    dfCustDay.col("TX_DATE"),\
                    zeroifnull(dfCustTrxFraud.col("TX_AMOUNT")).as_("TX_AMOUNT"),\
                    iff(col("TX_AMOUNT") > lit(0), lit(1), lit(0)).as_("NO_TRX"))\
                .groupBy(col("CUSTOMER_ID"), col("TX_DATE"))\
                .agg([sum(col("TX_AMOUNT")).as_("TOT_AMOUNT"), sum(col("NO_TRX")).as_("NO_TRX")])

Now when we have the number of transactions and amount by customer and day we can aggregate by our windows (1, 7 and 30 days).

For getting values previous day we will use the **lag** function since it's only one value we want and for 7 and 30 days we will use sum over a window. Since we do not want to include future values we will not include the current day in the windows.

![Figure5](images/figure5.png)

In [ ]:
custDate = Window.partitionBy(col("customer_id")).orderBy(col("TX_DATE"))
win7dCust = custDate.rowsBetween(-7, -1)
win30dCust = custDate.rowsBetween(-30, -1)

dfCustFeatDay = dfCustTrxByDay.select(col("TX_DATE"),col("CUSTOMER_ID"),col("NO_TRX"),col("TOT_AMOUNT"),
                              lag(col("NO_TRX"),1).over(custDate).as_("CUST_TX_PREV_1"),
                              sum(col("NO_TRX")).over(win7dCust).as_("CUST_TX_PREV_7"),
                              sum(col("NO_TRX")).over(win30dCust).as_("CUST_TX_PREV_30"),
                              lag(col("TOT_AMOUNT"),1).over(custDate).as_("CUST_TOT_AMT_PREV_1"),
                              sum(col("TOT_AMOUNT")).over(win7dCust).as_("CUST_TOT_AMT_PREV_7"),
                              sum(col("TOT_AMOUNT")).over(win30dCust).as_("CUST_TOT_AMT_PREV_30"))

In [ ]:
dfCustFeatDay.show()

Now we know for each customer and day the number of transactions and amount for previous 1, 7 and 30 days and we add that to our transactions.

In this step we will also use a window function to count the number of transactions and total amount for the current date, in order to only include the previous transactions for the same date we will create a partion key that consists of transaction date and customer id. By using that we get a rolling sum of all previous rows that is for the same date and customer.

![Figure6](images/figure6.png)

In [ ]:
winCurrDate = Window.partitionBy(col("PARTITION_KEY")).orderBy(col("TX_DATETIME")).rangeBetween(Window.unboundedPreceding,Window.currentRow)

dfCustBehaviurFeat = dfDateTimeFeat.join(dfCustFeatDay, (dfDateTimeFeat.col("CUSTOMER_ID") == dfCustFeatDay.col("CUSTOMER_ID")) & \
                                             (to_date(dfDateTimeFeat.col("TX_DATETIME")) == dfCustFeatDay.col("TX_DATE")))\
        .withColumn("PARTITION_KEY", call_builtin("concat",dfDateTimeFeat.col("CUSTOMER_ID"), to_date(dfDateTimeFeat.col("TX_DATETIME"))))\
        .withColumns(["CUR_DAY_TRX",\
                         "CUR_DAY_AMT"],\
                      [count(dfDateTimeFeat.col("CUSTOMER_ID")).over(winCurrDate), \
                      sum(dfDateTimeFeat.col("TX_AMOUNT")).over(winCurrDate)])\
        .select(dfDateTimeFeat.col("TRANSACTION_ID"), \
                dfDateTimeFeat.col("CUSTOMER_ID").as_("CUSTOMER_ID"), \
                dfDateTimeFeat.col("TERMINAL_ID"),\
                dfDateTimeFeat.col("TX_DATETIME").as_("TX_DATETIME"), \
                dfDateTimeFeat.col("TX_AMOUNT"),\
                dfDateTimeFeat.col("TX_TIME_SECONDS"),\
                dfDateTimeFeat.col("TX_TIME_DAYS"), \
                dfDateTimeFeat.col("TX_FRAUD"),\
                dfDateTimeFeat.col("TX_FRAUD_SCENARIO"),\
                dfDateTimeFeat.col("TX_DURING_WEEKEND"), \
                dfDateTimeFeat.col("TX_DURING_NIGHT"),\
                (zeroifnull(dfCustFeatDay.col("CUST_TX_PREV_1")) + col("CUR_DAY_TRX")).as_("CUST_CNT_TX_1"),\
                (zeroifnull(dfCustFeatDay.col("CUST_TOT_AMT_PREV_1")) + col("CUR_DAY_AMT") / col("CUST_CNT_TX_1")).as_("CUST_AVG_AMOUNT_1"), \
                (zeroifnull(dfCustFeatDay.col("CUST_TX_PREV_7")) + col("CUR_DAY_TRX")).as_("CUST_CNT_TX_7"),\
                (zeroifnull(dfCustFeatDay.col("CUST_TOT_AMT_PREV_7")) + col("CUR_DAY_AMT") / col("CUST_CNT_TX_7")).as_("CUST_AVG_AMOUNT_7"),\
                (zeroifnull(dfCustFeatDay.col("CUST_TX_PREV_30")) + col("CUR_DAY_TRX")).as_("CUST_CNT_TX_30"),\
                (zeroifnull(dfCustFeatDay.col("CUST_TOT_AMT_PREV_30")) + col("CUR_DAY_AMT") / col("CUST_CNT_TX_30")).as_("CUST_AVG_AMOUNT_30"))



In [ ]:
dfCustBehaviurFeat.show()

### Terminal ID transformations

The main goal with the Terminal ID transformations will be to extract a risk score, that assesses the exposure of a given terminal ID to fraudulent transactions. The risk score will be defined as the average number of fraudulent transactions that occurred on a terminal ID over a time window. As for customer ID transformations, we will use three window sizes, of 1, 7, and 30 days.

Contrary to customer ID transformations, the time windows will not directly precede a given transaction. Instead, they will be shifted back by a delay period. The delay period accounts for the fact that, in practice, the fraudulent transactions are only discovered after a fraud investigation or a customer complaint. Hence, the fraudulent labels, which are needed to compute the risk score, are only available after this delay period. To a first approximation, this delay period will be set to one week.

Part from above the logic is rather similar to how we calculated for customer.

In [ ]:
# Since we aggregate by terminal and day and not all terminals have transactions for all dates we cross join our date dataframe with our terminal table so each terminal will have one row for each date
dfTerminals = session.table("TERMINALS").select("TERMINAL_ID")
dfTermDay = dfDays.join(dfTerminals)

# Aggregate number of transactions and amount by terminal and date, for dates where a terminal do not have any ttransactions we ad a 0
dfTermTrxByDay = dfCustTrxFraud.join(dfTermDay, (dfCustTrxFraud.col("TERMINAL_ID") == dfTermDay.col("TERMINAL_ID"))\
                    & (to_date(dfCustTrxFraud.col("TX_DATETIME")) == dfTermDay.col("TX_DATE")), "rightouter")\
                .select(dfTermDay.col("TERMINAL_ID").as_("TERMINAL_ID"),\
                        dfTermDay.col("TX_DATE"), \
                        zeroifnull(dfCustTrxFraud.col("TX_FRAUD")).as_("NB_FRAUD"), \
                        when(is_null(dfCustTrxFraud.col("TX_FRAUD")), lit(0)).otherwise(lit(1)).as_("NO_TRX")) \
                .groupBy(col("TERMINAL_ID"), col("TX_DATE"))\
                .agg([sum(col("NB_FRAUD")).as_("NB_FRAUD"), sum(col("NO_TRX")).as_("NO_TRX")])

# Aggregate by our windows.
termDate = Window.partitionBy(col("TERMINAL_ID")).orderBy(col("TX_DATE"))
winDelay = termDate.rowsBetween(-7, -1) 
win1dTerm = termDate.rowsBetween(-8, -1) # We need to add the Delay period to our windows
win7dTerm = termDate.rowsBetween(-14, -1)
win30dTerm = termDate.rowsBetween(-37, -1)

dfTermFeatDay = dfTermTrxByDay.select(col("TX_DATE"),col("TERMINAL_ID"),col("NO_TRX"),col("NB_FRAUD"),
                              sum(col("NB_FRAUD")).over(winDelay).as_("NB_FRAUD_DELAY"),
                              sum(col("NO_TRX")).over(winDelay).as_("NB_TX_DELAY"),
                              sum(col("NO_TRX")).over(win1dTerm).as_("NB_TX_DELAY_WINDOW_1"),
                              sum(col("NO_TRX")).over(win7dTerm).as_("NB_TX_DELAY_WINDOW_7"),
                              sum(col("NO_TRX")).over(win30dTerm).as_("NB_TX_DELAY_WINDOW_30"),
                              sum(col("NB_FRAUD")).over(win1dTerm).as_("NB_FRAUD_DELAY_WINDOW_1"),
                              sum(col("NB_FRAUD")).over(win7dTerm).as_("NB_FRAUD_DELAY_WINDOW_7"),
                              sum(col("NB_FRAUD")).over(win30dTerm).as_("NB_FRAUD_DELAY_WINDOW_30"))

dfTermBehaviurFeat = dfCustBehaviurFeat.join(dfTermFeatDay, (dfCustBehaviurFeat.col("TERMINAL_ID") == dfTermFeatDay.col("TERMINAL_ID")) &\
                                                 (to_date(dfDateTimeFeat.col("TX_DATETIME")) == dfTermFeatDay.col("TX_DATE")))\
            .withColumns(["PARTITION_KEY",\
                             "CUR_DAY_TRX",\
                             "CUR_DAY_FRAUD"],\
                         [call_builtin("concat", dfDateTimeFeat.col("TERMINAL_ID"), to_date(dfDateTimeFeat.col("TX_DATETIME"))),\
                             count(dfDateTimeFeat.col("TERMINAL_ID")).over(winCurrDate),\
                             sum(dfDateTimeFeat.col("TX_FRAUD")).over(winCurrDate)]\
                          )\
             .withColumns(["NB_TX_DELAY", \
                              "NB_FRAUD_DELAY",\
                              "NB_TX_DELAY_WINDOW_1",\
                              "NB_FRAUD_DELAY_WINDOW_1",\
                              "NB_TX_DELAY_WINDOW_7",\
                              "NB_FRAUD_DELAY_WINDOW_7",\
                              "NB_TX_DELAY_WINDOW_30",\
                              "NB_FRAUD_DELAY_WINDOW_30"],\
                           [dfTermFeatDay.col("NB_TX_DELAY") + col("CUR_DAY_TRX"),\
                               col("NB_FRAUD_DELAY") +  col("CUR_DAY_FRAUD"),\
                               col("NB_TX_DELAY_WINDOW_1") + col("CUR_DAY_TRX"),\
                               col("NB_FRAUD_DELAY_WINDOW_1") + col("CUR_DAY_FRAUD"),\
                               col("NB_TX_DELAY_WINDOW_7") + col("CUR_DAY_TRX"),\
                               col("NB_FRAUD_DELAY_WINDOW_7") + col("CUR_DAY_FRAUD"),\
                               col("NB_TX_DELAY_WINDOW_30") + col("CUR_DAY_TRX"),\
                               col("NB_FRAUD_DELAY_WINDOW_30") + col("CUR_DAY_FRAUD")])\
             .select(dfCustBehaviurFeat.col("TRANSACTION_ID"), \
                     dfCustBehaviurFeat.col("TX_DATETIME").as_("TX_DATETIME"),\
                     dfCustBehaviurFeat.col("CUSTOMER_ID").as_("CUSTOMER_ID"), \
                     dfCustBehaviurFeat.col("TERMINAL_ID").as_("TERMINAL_ID"),\
                     dfCustBehaviurFeat.col("TX_TIME_SECONDS"), \
                     dfCustBehaviurFeat.col("TX_TIME_DAYS"), \
                     dfCustBehaviurFeat.col("TX_AMOUNT"), \
                     dfCustBehaviurFeat.col("TX_FRAUD"), \
                     dfCustBehaviurFeat.col("TX_FRAUD_SCENARIO"),\
                     dfCustBehaviurFeat.col("TX_DURING_WEEKEND"), \
                     dfCustBehaviurFeat.col("TX_DURING_NIGHT"),\
                     dfCustBehaviurFeat.col("CUST_AVG_AMOUNT_1"),\
                     dfCustBehaviurFeat.col("CUST_CNT_TX_1"), \
                     dfCustBehaviurFeat.col("CUST_AVG_AMOUNT_7"),\
                     dfCustBehaviurFeat.col("CUST_CNT_TX_7"), \
                     dfCustBehaviurFeat.col("CUST_AVG_AMOUNT_30"),\
                     dfCustBehaviurFeat.col("CUST_CNT_TX_30"),\
                     (col("NB_TX_DELAY_WINDOW_1") - col("NB_TX_DELAY")).as_("NB_TX_WINDOW_1"),\
                     iff(col("NB_FRAUD_DELAY_WINDOW_1") - col("NB_FRAUD_DELAY") > 0, \
                             (col("NB_FRAUD_DELAY_WINDOW_1") - col("NB_FRAUD_DELAY")) / col("NB_TX_WINDOW_1"), lit(0)).as_("TERM_RISK_1"),\
                     (col("NB_TX_DELAY_WINDOW_7") - col("NB_TX_DELAY")).as_("NB_TX_WINDOW_7"),\
                     iff(col("NB_FRAUD_DELAY_WINDOW_7") - col("NB_FRAUD_DELAY") > 0, \
                             (col("NB_FRAUD_DELAY_WINDOW_7") - col("NB_FRAUD_DELAY"))  / col("NB_TX_WINDOW_7"), lit(0)).as_("TERM_RISK_7"),\
                     (col("NB_TX_DELAY_WINDOW_30") - col("NB_TX_DELAY")).as_("NB_TX_WINDOW_30"),\
                     iff(col("NB_FRAUD_DELAY_WINDOW_30") - col("NB_FRAUD_DELAY") > 0, \
                             (col("NB_FRAUD_DELAY_WINDOW_30") - col("NB_FRAUD_DELAY"))  / col("NB_TX_WINDOW_30"), lit(0)).as_("TERM_RISK_30"))

We now have our new features and can save it into a new table that we then can use for traing our model for predicting fraud.

Start by checking the schema of our new dataframe and decide if we will keep all columns.

In [ ]:
dfTermBehaviurFeat.schema

As mentioned we are not actually executing anything in Snowflake, unless when using .show(), and has just created a execution plan.

We can get the execution plan and SQL needed to prefrom all our steps by calling **explain**

In [ ]:
dfTermBehaviurFeat.explain()

We can quicly scale up our Warhouse (compute) before we load our new table and after we done the load we can scale it down again.

By creating a function for it we can simplify the code and make it easier for the developers.

In [ ]:
def scaleWh(sess, wh, size):
    if (len(wh) == 0): 
        return false
    if (len(size) == 0):
        return false
   
    alterSQL = "ALTER WAREHOUSE " + wh + " SET WAREHOUSE_SIZE = " + size
    sess.sql(alterSQL).collect()
    return True


In [ ]:
scaleWh(session, "COMPUTE_WH", "LARGE")

We we run **saveAsTable** Snowpark will generate the SQL for all the previous step, execute it in Snowflake and store the result in the table **customer_trx_fraud_features**.

In [ ]:
dfTermBehaviurFeat.write.mode("overwrite").saveAsTable("customer_trx_fraud_features")

In [ ]:
scaleWh(session, "COMPUTE_WH", "SMALL")